In [1]:
from main_fullvision import Langda_Agent
from config import paths
addition = {}
addition["user_context"] = """
please notice distance(X, Y) is a value, so please do not create single clause like: distance(X, bomb) > 10.
please seen initial_charge, charge_cost and weight as value and use in following form:
V is value, value < 5 or value > 10...
"""
rules_string = ""
rule_path = paths.get_abscase_path("rules/test/addition_prompt.pl")
with open (rule_path,"r") as f:
    rules_string += f.read()

l = Langda_Agent(rules_string, "deepseek-chat")
l.call_langda_workflow()



processing parse_node...
getting codes with comments...
processing langda and lann terms...
processing _decide_next_init ...
processing generate_node...
processing _decide_next_gnrt... #current round: 1
processing test_node...
origin_fest_codes [{'C3C3CD7C': None}, {'CECE1133': None}]
new_fest_codes [{'C3C3CD7C': '% === % LLM Generated Logic Codes % === %\ndigit(X,Y) :- between(0,9,Y).\n% === % ========================= % === %'}]


IndexError: list index out of range

In [75]:
from typing import List
from problog.program import PrologString
from config import paths
import re
import json
from problog.program import PrologString, Clause, AnnotatedDisjunction, Term

def pretty_print_prolog(program_str: str) -> str:
    """
    格式化Prolog代码，添加适当的换行和缩进
    1) 拆分：按非小数点句末的句点分割多条语句；
    2) 解析：对每条语句用 PrologString 生成 AST；
    3) 重排：根据 AST 类型做换行与缩进。
    """
    # —— 1. 预处理：去除换行并压缩空白 ——  
    flat = ' '.join(program_str.split())
    
    # —— 2. 用正则拆分独立语句 ——
    # 改进的正则表达式，更精确地处理句点
    # 这个正则表达式需要考虑到小数点是数字的一部分，而非句子的结束
    parts = []
    current = ""
    i = 0
    while i < len(flat):
        if flat[i] == '.':
            # 检查前后是否为数字，若不是则视为语句结束
            is_sentence_end = False
            if i > 0 and not flat[i-1].isdigit() and (i == len(flat)-1 or not flat[i+1].isdigit()):
                is_sentence_end = True
            
            current += flat[i]
            if is_sentence_end:
                parts.append(current.strip())
                current = ""
        else:
            current += flat[i]
        i += 1
    
    if current.strip():  # 处理最后可能没有句点的语句
        parts.append(current.strip())
    
    # 调试输出
    print(f"拆分后的语句: {parts}")
    
    lines = []
    def flatten_body(term: Term) -> list[str]:
        if isinstance(term, Term) and term.functor == ',' and len(term.args) == 2:
            return flatten_body(term.args[0]) + flatten_body(term.args[1])
        return [str(term)]

    # —— 3. 对每条语句分别解析、格式化 ——  
    for part in parts:
        stmt_str = part.strip()
        if not stmt_str.endswith('.'):
            stmt_str += '.'
            
        try:
            prog = PrologString(stmt_str)  # AST 生成
            found_valid_stmt = False
            
            for stmt in prog:
                found_valid_stmt = True
                if isinstance(stmt, Clause):
                    head = str(stmt.head)
                    atoms = flatten_body(stmt.body)
                    if atoms == ['true']:
                        lines.append(f"{head}.")
                    else:
                        lines.append(f"{head} :-")
                        for a in atoms[:-1]:
                            lines.append(f"    {a},")
                        lines.append(f"    {atoms[-1]}.")
                elif isinstance(stmt, AnnotatedDisjunction):
                    # 处理带概率的断言
                    for branch in stmt.heads:
                        lines.append(f"{branch.probability}::{branch} :- true.")
                else:
                    # query/evidence/directive 等
                    lines.append(f"{stmt}.")
                    
            if not found_valid_stmt:
                print(f"未找到有效语句: {stmt_str}")
                lines.append(stmt_str)
                
        except Exception as e:
            # 若解析失败，记录错误并原样输出
            print(f"解析失败: {stmt_str}, 错误: {str(e)}")
            lines.append(stmt_str)

    # 调试输出
    print(f"格式化后的行数: {len(lines)}")
    formatted = "\n".join(lines)
    print(f"格式化的前100个字符: {formatted[:100]}")
    
    return formatted

def _find_all_blocks(type, text:str, ext_pattern:str="") -> List[dict]:
    """
    find block with certain patterns in json form.
    Args:
        type: find block in code or in report file, if choose "other", it will use external pattern
        text: the text that needs to search through
        other_pattern: use other patterns
    """
    blocks:List[dict] = []
    print("inside")
    if type == "code":
        """
        return with [{"hash":"code content",...] form
        """
        print("inside code")
        # pattern = r"```(?:prolog|[a-z]*)?\n(\{[\s\S]*?\})```"
        pattern = r"```(?:prolog|[a-z]*)?\n(.*?)```"
        matches = re.findall(pattern, text, re.DOTALL)
        # print("matches",len(matches),matches)
        if matches:
            for code in matches:
                try:
                    code_block = json.loads(code.replace("\n", ""))
                    # print("block",code_block)
                    program = pretty_print_prolog(code_block["Code"])
                    with open(paths.get_abscase_path("program"),"w") as f:
                        f.write(program)

                    code_block["Code"] = "% === % LLM Generated Logic Codes % === %\n" + program + "\n% === % ========================= % === %"
                    blocks.append({code_block["HASH"]:code_block["Code"]})
                except ValueError:
                    print(f"_find_all_blocks: could not parse the code block of {code_block}")

    elif type == "report":
        """
        return with [{"hash":{"HASH":"hash","Report":"report content","Need_regenerate":"True"}},...] form
        """
        pattern = r"```report\s+(.*?)```"
        matches = re.findall(pattern, text, re.DOTALL)
        if matches:
            for report in matches:
                try:
                    block = json.loads(report)
                    blocks.append({block["HASH"]:block})
                except ValueError:
                    blocks.append({block["HASH"]:None})

    else:
        raise ValueError("you must choose from ['report','code','other']")
    return blocks

if __name__ == "__main__":
    line = """
    ```prolog
    {
        "HASH": "EC80EE82",
        "Code": "rnn(Text,Embedding),
    net1(Embedding,P),
    permute(P,X1,X2,X3,A,B,C),
    min(A,B,Min1),
    max(A,B,Max1),
    min(Max1,C,Min2),
    max(Max1,C,Max2),
    operator(plus,Min1,Min2,Sum),
    operator(minus,Sum,Max2,Out)."
    }
    ```

    """
    txx = """
    coin(c1). coin(c2).0.4::heads(C);0.6::tails(C):-coin(C). win:-heads(c1),heads(c2).evidence(heads(c2),false).
    """
    print(_find_all_blocks("code",line))
    # print(pretty_print_prolog(txx))


inside
inside code
拆分后的语句: ['rnn(Text,Embedding), net1(Embedding,P), permute(P,X1,X2,X3,A,B,C), min(A,B,Min1), max(A,B,Max1), min(Max1,C,Min2), max(Max1,C,Max2), operator(plus,Min1,Min2,Sum), operator(minus,Sum,Max2,Out).']
格式化后的行数: 1
格式化的前100个字符: rnn(Text,Embedding), net1(Embedding,P), permute(P,X1,X2,X3,A,B,C), min(A,B,Min1), max(A,B,Max1), min
[{'EC80EE82': '% === % LLM Generated Logic Codes % === %\nrnn(Text,Embedding), net1(Embedding,P), permute(P,X1,X2,X3,A,B,C), min(A,B,Min1), max(A,B,Max1), min(Max1,C,Min2), max(Max1,C,Max2), operator(plus,Min1,Min2,Sum), operator(minus,Sum,Max2,Out).\n% === % ========================= % === %'}]


In [81]:
import re

# s = '''{
#     "HASH": "EC80EE82",
#     "Code": "rnn(Text,Embedding),
# net1(Embedding,P),
# permute(P,X1,X2,X3,A,B,C),
# min(A,B,Min1),
# max(A,B,Max1),
# min(Max1,C,Min2),
# max(Max1,C,Max2),
# operator(plus,Min1,Min2,Sum),
# operator(minus,Sum,Max2,Out)."
# }'''
s = """
```prolog
{
    "HASH": "214DEF87",
    "Code": "game(Cards,House,Outcome) :-\n    cards(Cards,own,House,C1),\n    cards(Cards,opponent,House,C2),\n    best_hand_rank(C1,R1),\n    best_hand_rank(C2,R2),\n    outcome(R1,R2,Outcome)."
}
```
```prolog
{
    "HASH": "475C8589",
    "Code": "almost_equal(X,Y) :- ground(Y), abs(X-Y) < 0.0001.\nalmost_equal(X,Y) :- var(Y), Y is float(X)."
}
```
```prolog
{
    "HASH": "EC80EE82",
    "Code": "rnn(Text,Embedding),
net1(Embedding,P),
permute(P,X1,X2,X3,A,B,C),
min(A,B,Min1),
max(A,B,Max1),
min(Max1,C,Min2),
max(Max1,C,Max2),
operator(plus,Min1,Min2,Sum),
operator(minus,Sum,Max2,Out)."
}
```
"""

def _find_all_blocks(type, text:str, ext_pattern:str="") -> List[dict]:
    """
    find block with certain patterns in json form.
    Args:
        type: find block in code or in report file, if choose "other", it will use external pattern
        text: the text that needs to search through
        other_pattern: use other patterns
    """
    blocks:List[dict] = []
    if type == "other":
        """
        return with [block,...] form
        """
        blocks:List[str] = []
        pattern = ext_pattern
        matches = re.findall(pattern, text, re.DOTALL)
        if matches:
            for code in matches:
                try:
                    block = json.loads(code)
                    blocks.append(block)
                except ValueError:
                    blocks.append(None)

    elif type == "code":
        """
        return with [{"hash":"code content",...] form
        """

        pattern = r"```(?:prolog|[a-z]*)?\n(.*?)```"
        matches = re.findall(pattern, text, re.DOTALL)
        if matches:
            for code in matches:
            #     try:
            #         block = json.loads(code)
            #         block["Code"] = "% === % LLM Generated Logic Codes % === %\n"+block["Code"]+"\n% === % ========================= % === %"
            #         blocks.append({block["HASH"]:block["Code"]})
            #     except ValueError:
            #         print(f"_find_all_blocks: could not parse the code block of {block}")
            #         blocks.append({block["HASH"]:None})
                try:
                    m_hash = re.search(r'"HASH"\s*:\s*"([^"]*)"', code)
                    hash_val = m_hash.group(1) if m_hash else None

                    # 2. 提取 Code 整段
                    m_code = re.search(r'"Code"\s*:\s*"([\s\S]*?)"', code)
                    code_block = m_code.group(1) if m_code else ""
                    blocks.append({hash_val:code_block})

                except ValueError as e:
                    print(e)
        
    elif type == "report":
        """
        return with [{"hash":{"HASH":"hash","Report":"report content","Need_regenerate":"True"}},...] form
        """
        pattern = r"```report\s+(.*?)```"
        matches = re.findall(pattern, text, re.DOTALL)
        if matches:
            for report in matches:
                try:
                    block = json.loads(report)
                    blocks.append({block["HASH"]:block})
                except ValueError:
                    blocks.append({block["HASH"]:None})

    else:
        raise ValueError("you must choose from ['report','code','other']")
    return blocks

_find_all_blocks("code",s)

[{'214DEF87': 'game(Cards,House,Outcome) :-\n    cards(Cards,own,House,C1),\n    cards(Cards,opponent,House,C2),\n    best_hand_rank(C1,R1),\n    best_hand_rank(C2,R2),\n    outcome(R1,R2,Outcome).'},
 {'475C8589': 'almost_equal(X,Y) :- ground(Y), abs(X-Y) < 0.0001.\nalmost_equal(X,Y) :- var(Y), Y is float(X).'},
 {'EC80EE82': 'rnn(Text,Embedding),\nnet1(Embedding,P),\npermute(P,X1,X2,X3,A,B,C),\nmin(A,B,Min1),\nmax(A,B,Max1),\nmin(Max1,C,Min2),\nmax(Max1,C,Max2),\noperator(plus,Min1,Min2,Sum),\noperator(minus,Sum,Max2,Out).'}]

In [23]:
langda_ext_pattern = r'/\*\s*(\w+)\s*\*/'

import re
langda = re.match(langda_ext_pattern, """/*)""")
if langda is not None:
    print(langda[1])

In [13]:
def process_msg_from_bot(langda_ext:str) -> dict:
    """
    I suppose the message from telegram bot is:
    from telegram: "!Secure: content about secure..."   
        ==> inside langda file: langda(LLM:"/* Secure */")  
        ==> langda(LLM:"content about secure..")
    """
    bot_pattern = r'!(\w+):\s*(.+)'
    matches = re.findall(bot_pattern, langda_ext)
    result_dict = dict(matches)

    return result_dict

print(process_msg_from_bot(
    """
    !Secure: Stay away from bomb! 
    !Note: Stop that! It's insecure. who are you
    hi there
    """))


{'Secure': 'Stay away from bomb! ', 'Note': "Stop that! It's insecure. who are you"}
